In [ ]:
!nvidia-smi

Fri Jun 17 22:55:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers

#Setup and Config

In [ ]:
!gdown --id 1ZGIHE4DF1Nb316jPBWyU7Zzp_K1Lp5xM

In [ ]:
!unzip "/content/TaskA-Memes.zip"
!rm TaskA-Memes.zip

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import seaborn as sns

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
seed = 41

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Data

In [ ]:
df = pd.read_csv('/content/TaskA-Memes/task_a.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df = df[df.meme_type != 'only offensive']

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df['hate_speech'] = [1 if meme == 'hate_speech' else 0 for meme in df.meme_type]
df.drop(columns='meme_type', inplace=True)

In [ ]:
df

,file_name,text,hate_speech
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,1
2,img10.jpg,لا أحد النسويات,1
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,1
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,1
...,...,...,...
1669,IMG_269.jpg,لاتوجد نساء اجمل منك مهما بحثت والله؟! يعني بح...,0
1670,IMG_233.jpg,أنا أخوض حرب لا أعلم عنها شيئا,0
1671,IMG_138.jpg,لما اروح عند دكتور الاسنان يادكتور بالله عليك ...,0
1672,IMG_705.jpg,الضربة اللي متموتش تقوي لأ انا عايزاللي تموت,0


In [ ]:
train_images = df.file_name
train_text = df.text
train_labels = df.hate_speech

In [ ]:
train_data = []

for i in range(len(df)):
  train_data.append([train_images[i], train_text[i], train_labels[i]])

In [ ]:
train_data = pd.DataFrame(train_data, columns=['images','text','labels'])
train_data

,images,text,labels
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,1
2,img10.jpg,لا أحد النسويات,1
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,1
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,1
...,...,...,...
1669,IMG_269.jpg,لاتوجد نساء اجمل منك مهما بحثت والله؟! يعني بح...,0
1670,IMG_233.jpg,أنا أخوض حرب لا أعلم عنها شيئا,0
1671,IMG_138.jpg,لما اروح عند دكتور الاسنان يادكتور بالله عليك ...,0
1672,IMG_705.jpg,الضربة اللي متموتش تقوي لأ انا عايزاللي تموت,0


In [ ]:
train_df, test_df = train_test_split(train_data, test_size=0.1, random_state=seed)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
print(len(train_df))
print(len(test_df))

1506
168


In [ ]:
train_df

,images,text,labels
52,img44.jpg,عندما تعلم ان السحاب هو ما ينزل المطر وليس الل...,1
1006,IMG_884.jpg,إنت لوعليك اقساط مش هتصحى من النوم مكشركده !,0
25,img33.jpg,مسلم كيوت لا تخافوا الكل سيدخل الجنة,1
208,img242.png,زلمة أنا برأيي أنا,1
869,img1223.jpg,كيف نتخيل الاعمى يرى العالم ماذا يرى الاعمى با...,1
...,...,...,...
1625,IMG_771.jpg,الي بيحصل الفترة دي انا,0
243,img268.png,الناس الي بتفكر انه النسوية هي كره للرجال,1
321,img326.png,بتسمعه بقولوا بالشارع البنات بعرفش يسوقوا منيح,1
1104,IMG_426.jpg,الناس كلها عطشانة وعايزة تشرب مياه انا : جعان ...,0


In [ ]:
test_df

,images,text,labels
708,img1044.jpg,الرب يخلق اليهود الرب ايضا يخلق عيسى اليهود يق...,1
31,img22.jpg,لما الضيوف يقعدوا ساكتين لربك نبعد مولين جماعة...,0
472,img501.jpg,شبكة نتفليكس عمل جيد لكن لماذا هذا الفيلم لا ي...,1
1360,IMG_920.jpg,2019 :شخص كسول لا فائدة منه 2020:مواطن مالح و ...,0
660,img1013.png,المسلم بي لايك النظرة الاولى حلال,1
...,...,...,...
1562,IMG_1012.jpg,الحلاقين حاليا,0
1405,IMG_498.jpg,لما ارتبط ب واحد مش بتاع بنات وبيصلي بيحبني ان...,0
206,img245.png,حقوق أساسية للمرأة الزلام هل هذه سيابة,1
522,img544.jpg,فرنسا تنهب ثروات الشعوب الافريقية وتتسبب تفقير...,1


# Model

In [ ]:
model_args = ClassificationArgs(
    num_train_epochs=1,
    fp16=False,
    train_batch_size=64,
    learning_rate=1e-5,
    max_seq_length=32,
    save_best_model=True,
    overwrite_output_dir=True,
    save_model_every_epoch=False,
    gradient_accumulation_steps=1,
    labels_list=[0,1]
)

In [ ]:
model = ClassificationModel(
  'bert',
  'aubmindlab/bert-base-arabertv02',
  use_cuda=True,
  num_labels=2,
  args=model_args,
) 

In [ ]:
model.train_model(train_df)

In [ ]:
import sklearn

result, model_outputs, wrong_predictions  = model.eval_model(
    test_df,
    acc=sklearn.metrics.accuracy_score,
)

In [ ]:
# model_outputs

In [ ]:
test_df

,images,text,labels
708,img1044.jpg,الرب يخلق اليهود الرب ايضا يخلق عيسى اليهود يق...,1
31,img22.jpg,لما الضيوف يقعدوا ساكتين لربك نبعد مولين جماعة...,0
472,img501.jpg,شبكة نتفليكس عمل جيد لكن لماذا هذا الفيلم لا ي...,1
1360,IMG_920.jpg,2019 :شخص كسول لا فائدة منه 2020:مواطن مالح و ...,0
660,img1013.png,المسلم بي لايك النظرة الاولى حلال,1
...,...,...,...
1562,IMG_1012.jpg,الحلاقين حاليا,0
1405,IMG_498.jpg,لما ارتبط ب واحد مش بتاع بنات وبيصلي بيحبني ان...,0
206,img245.png,حقوق أساسية للمرأة الزلام هل هذه سيابة,1
522,img544.jpg,فرنسا تنهب ثروات الشعوب الافريقية وتتسبب تفقير...,1


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

_, preds = torch.max(torch.tensor(model_outputs), dim=1)
labels = test_df.labels

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))